<a href="https://colab.research.google.com/github/beangoben/TB_project/blob/master/mordredFingerprinting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [13]:
!wget https://raw.githubusercontent.com/beangoben/rdkit_colab/master/colab_utils.py -O colab_utils.py
!rm -rf sample_data
import colab_utils

--2020-08-06 19:37:36--  https://raw.githubusercontent.com/beangoben/rdkit_colab/master/colab_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7173 (7.0K) [text/plain]
Saving to: ‘colab_utils.py’

colab_utils.py      100%[===================>]   7.00K  --.-KB/s    in 0s      

2020-08-06 19:37:36 (43.7 MB/s) - ‘colab_utils.py’ saved [7173/7173]



In [14]:
github_repo = 'git@github.com:beangoben/TB_project.git'
# If working in a private colab repo, 
# save a id_rsa file in gdrive or dropbox, etc. and run copy_ssh_key
colab_utils.copy_ssh_key('https://www.dropbox.com/s/h6aw94l7sz763sz/id_rsa_colab?dl=1')
colab_utils.clone_repo(github_repo)
colab_utils.conda_install_from_yaml()
colab_utils.pip_install_from_yaml()

CMD: git clone --recursive git@github.com:beangoben/TB_project.git
Ignoring: conda install already exists in {CONDA_DIR}!
pip installing []


In [15]:
colab_utils.add_conda_dir_to_python_path()
import sys
REPO_DIR = '.' if 'google.colab' in sys.modules else '..'

In [16]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import scipy as sp
import scipy.stats as stats
import sklearn
import sklearn.model_selection
import sklearn.metrics
import sklearn.cluster
import pandas as pd
import numpy as np
from IPython.display import display, HTML 
np.set_printoptions(threshold=sys.maxsize)
import umap
from mordred import Calculator, descriptors

# utils
from tqdm.autonotebook import tqdm
from collections import OrderedDict
import itertools

# rdkit stuff
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.SimDivFilters.rdSimDivPickers import MaxMinPicker

tqdm.pandas()
sns.set_context('talk', font_scale=1.5)
sns.set_style("white")
plt.rc('lines', linewidth=2)
plt.rc('figure', figsize=(10,5))

Forming Dataset with Mordred Smiles

In [17]:
mordred = pd.read_csv('/content/drive/My Drive/Extracurricular/Internships/Research/TB Internship/Datasets/mordred(1).csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,157,158,159,166,167,168,175,176,177,184,185,186,193,194,195,202,203,204,211,212,213,220,221,222,229,230,231,238,239,240,247,248,249,364,365,366,373,374,375,382,383,384,391,392,393,400,401,402,409,410,411,418,419,420,427,428,429,436,437,438,445,446,447,454,455,456,463,464,465,471,472,473,479,480,481,487,488,489,495,496,497,503,504,505,511,512,513,519,520,521,527,528,529,535,536,537,543,544,545,551,552,553,559,560,561,567,568,569,575,576,577,583,584,585,591,592,593,599,600,601,607,608,609,615,616,617,623,624,625,631,632,633,639,640,641,647,648,649,655,656,657,849,865,1072,1074,1076,1077,1078,1080,1081,1082,1083,1084,1086,1089,1090,1094,1095,1097,1099,1100,1101,1102,1103,1111,1115,1116,1118,1119,1151,1153,1155,1156,1157,1159,1160,1161,1162,1163,1165,1168,1169,1173,1174,1176,1178,1179,1180,1181,1182,1190,1194,1195,1197,1198,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,

In [ ]:
FullMerged = pd.read_csv('/content/drive/My Drive/Extracurricular/Internships/Research/TB Internship/FullMerged.csv')

In [ ]:
partial = FullMerged

In [ ]:
fplist = []
smileslist = []
calc = Calculator(descriptors, ignore_3D=True)
for smile in tqdm(partial.SMILES.unique()):
  mol = Chem.MolFromSmiles(smile)
  fp = calculation = calc(mol)[:]
  fplist.append(fp)
  smileslist.append(smile)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [ ]:
result = zip(smileslist, fplist)
resultdf = pd.DataFrame(result)
resultdf.rename(columns={0:'SMILES',
                          1:'fingerprint'}, 
                 inplace=True)

In [ ]:
fpdf = pd.DataFrame(resultdf.fingerprint.to_list())
fpdf['SMILES'] = smileslist
resultdf = resultdf.merge(fpdf)
resultdf = resultdf.drop(['fingerprint'], axis=1)

In [ ]:
FullMerged = FullMerged.merge(resultdf)